In [7]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from sklearn import metrics 
dat = yf.Ticker("NVDA")
df = yf.download(["NVDA"], period='120mo', interval='1d') # Por hora o por minuto, o por dia 
df[('Prev Close', 'NVDA')] = df[('Close', 'NVDA')].shift(1)  # Shifted by 1 to use previous day's close to predict today's
df.dropna(inplace=True)  # Remove rows with NaN values
df.tail()

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume,Prev Close
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA,NVDA
Date,,,,,,
2025-04-08,96.300003,105.849998,94.459999,103.809998,476243400,97.639999
2025-04-09,114.330002,115.099998,97.529999,98.889999,612918300,96.300003
2025-04-10,107.570000,110.860001,99.150002,109.370003,437812400,114.330002
2025-04-11,110.930000,111.550003,107.480003,108.500000,312853800,107.570000
2025-04-14,110.709999,114.290001,109.070000,114.139999,259339786,110.930000


In [8]:
import pyodbc 


try:
    with pyodbc.connect(r'DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;DATABASE=stock_market;Trusted_Connection=yes;')  as conn: # Si se pierde la conexion, cierra la sesion
        cursor = conn.cursor()
        # Method 2: Using itertuples() - faster than iterrows()
        for row in df.itertuples():
            print(f"Index: {row.Index}")
            print(row)
            
            data = {
                'ticker': 'NVDA',
                'date': row.Index,
                'close': row[1],
                'high': row[2],
                'low': row[3],
                'open': row[4],
                'volume': row[5]
            }
            print(data)
            cursor.execute("""
                INSERT INTO StockPrices (Ticker, "Date", "Close", "High", "Low", "Open", Volume)
                SELECT ?, ?, ?, ?, ?, ?, ?
                WHERE NOT EXISTS (
                    SELECT 1 FROM StockPrices 
                    WHERE Ticker = ? AND Date = ?
                )
            """, 
            data['ticker'], data['date'], data['close'], data['high'],
            data['low'], data['open'], data['volume'],
            data['ticker'], data['date'])

            cursor.execute("""
            UPDATE StockPrices
            SET     
                "Close" = COALESCE("Close", ?),
                "High" = COALESCE("High", ?),
                "Low" = COALESCE("Low", ?),
                "Open" = COALESCE("Open", ?),
                "Volume" = COALESCE("Volume", ?)
            WHERE Ticker = ? AND "Date" = ?
            """, data['close'], data['high'],
            data['low'], data['open'], data['volume'], data['ticker'], data["date"])
        
            if cursor.rowcount > 0:
                print("New record inserted successfully")
            else:
                print("Record already exists - no insertion made")
            
except pyodbc.Error as e:
    print(f"Database error: {e}")

Index: 2015-04-16 00:00:00
Pandas(Index=Timestamp('2015-04-16 00:00:00'), _1=0.5418148040771484, _2=0.5466330723732895, _3=0.5379601779525778, _4=0.5420557002604688, _5=155284000, _6=0.545187771320343)
{'ticker': 'NVDA', 'date': Timestamp('2015-04-16 00:00:00'), 'close': 0.5418148040771484, 'high': 0.5466330723732895, 'low': 0.5379601779525778, 'open': 0.5420557002604688, 'volume': 155284000}
New record inserted successfully
Index: 2015-04-17 00:00:00
Pandas(Index=Timestamp('2015-04-17 00:00:00'), _1=0.5348284840583801, _2=0.5401286040050998, _3=0.5302511103515909, _4=0.5304920066187981, _5=293636000, _6=0.5418148040771484)
{'ticker': 'NVDA', 'date': Timestamp('2015-04-17 00:00:00'), 'close': 0.5348284840583801, 'high': 0.5401286040050998, 'low': 0.5302511103515909, 'open': 0.5304920066187981, 'volume': 293636000}
New record inserted successfully
Index: 2015-04-20 00:00:00
Pandas(Index=Timestamp('2015-04-20 00:00:00'), _1=0.5321783423423767, _2=0.5406102843275368, _3=0.531214642596961,

In [9]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Step 1: Create target - tomorrow's close
df['Target'] = df[('Close', 'NVDA')].shift(-1)
df.dropna(inplace=True)  # Remove the last row with NaN target

# Step 2: Feature engineering - basic features
df['Return'] = df[('Close', 'NVDA')].pct_change()
df['Volatility'] = df['Return'].rolling(window=5).std()
df['Volume_Change'] = df[('Volume', 'NVDA')].pct_change()
df['Price_Range'] = df[('High', 'NVDA')] - df[('Low', 'NVDA')]

df.dropna(inplace=True)

# Features and target
features = ['Return', 'Volatility', 'Volume_Change', 'Price_Range']
X = df[features]
y = df['Target']

# Step 3: Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

# Step 4: Train XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
model.fit(X_train, y_train)

# Step 5: Evaluate model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

# Step 6: Predict tomorrow’s close
latest = df[features].iloc[-1:]
predicted_close = model.predict(latest)
print(f"Predicted close for tomorrow: ${predicted_close[0]:.2f}")

Mean Squared Error: 5305.110069462803
R^2 Score: -2.467185697854278
Predicted close for tomorrow: $17.54


In [10]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Step 1: Create target - tomorrow's close
df['Target'] = df[('Close', 'NVDA')].shift(-1)
df.dropna(inplace=True)  # Remove the last row with NaN target

# Step 2: Feature engineering - basic features
df['Return'] = df[('Close', 'NVDA')].pct_change()
df['Volatility'] = df['Return'].rolling(window=5).std()
df['Volume_Change'] = df[('Volume', 'NVDA')].pct_change()
df['Price_Range'] = df[('High', 'NVDA')] - df[('Low', 'NVDA')]

df.dropna(inplace=True)

# Features and target
features = ['Return', 'Volatility', 'Volume_Change', 'Price_Range']
X = df[features]
y = df['Target']

# Step 3: Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

# Step 4: Train XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
model.fit(X_train, y_train)

# Step 5: Evaluate model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

# Step 6: Predict tomorrow’s close
latest = df[features].iloc[-1:]
predicted_close = model.predict(latest)
print(f"Predicted close for tomorrow: ${predicted_close[0]:.2f}")


Mean Squared Error: 5310.964717941794
R^2 Score: -2.4667569384189783
Predicted close for tomorrow: $17.77
